In [33]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append("../")
from src.data import *

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb

In [34]:
X, y = load_data()

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

$Preproccesing data for XGBoost and LightGBM$

In [36]:
wp = WellProcessor()

In [37]:
wp.fit(X_train)
X_train = wp.transform(X_train)

In [38]:
X_test = wp.transform(X_test)

$XGBoost$

Becuase it is one of the strongest, if not the strongest, model on the market currently. I wanted to start with XGBoost to see what kind of results it can give as a benchmark.

In [39]:
# Instantiate XGBClassifier
XGB = XGBClassifier()

# Fit XGBClassifier
XGB.fit(X_train, y_train)

# Predict on training and test sets
training_preds = XGB.predict(X_train)
test_preds = XGB.predict(X_test)

While XGBoost gave great results with default hyperparameters, it took a minimum of 12minutes to run. I wanted to compare its results to more efficent models.

*$DecisionTreeClassifier$*

My next model was a Decision Tree. I wanted something to compare to XGBoost that was a little bit more simple, but still could deal with complex models. I also wanted a model that could execute faster than XGBoost.

In [4]:
rn_dtc = run_model (X_train, y_train, DecisionTreeClassifier())

0.7634231192054719


In [5]:
rn_dtc.best_score_

0.7634231192054719

We had a nice score from our Decision Tree Classifier, but not as high as our XGBoost.

$Random Forest Classifier$

While my Decision Tree Classifier gave good results I wanted to try a more complex Random Forest Classifier to compare it to. I had great success with Random Forest in previous work, so it seemed like a natural choice.

In [15]:
rn_rfc = run_model (X_train, y_train, RandomForestClassifier())

0.7828733730297197


In [16]:
rn_rfc.best_score_

0.7828733730297197

My Random Forest gave a better score than my Decision Tree, which was great but the improvement wasnt as large as I was hoping. It was also slower than my Decision Tree.

$Naive Bayes$

While the Random Forest was a success, I wanted to see if we can improve a scores by a greater margin. One of the biggest challnenges for our data is the dataset is so large. Naive Bayes models are recorded to be better with larger datasets, so I wanted to use one next.

In [9]:
rn_gnb = run_model (X_train, y_train, GaussianNB())

0.1799257700085921


In [10]:
rn_gnb.best_score_

0.1799257700085921

Huge drop in score, the dream has ended, hope is gone.

$LightGBM$

After a misreble attempt with a Naive Bayes model I did some research into how to make our current best model even better. LightGBM is reported to be as competitive as XGBoost but with much faster times.

In [23]:
rn_lgb = run_model (X_train, y_train, lgb)

TypeError: Last step of Pipeline should implement fit or be the string 'passthrough'. '<module 'lightgbm' from '/opt/anaconda3/lib/python3.8/site-packages/lightgbm/__init__.py'>' (type <class 'module'>) doesn't